In [1]:
import msgpack
import boto3
import awswrangler as wr
from io import BytesIO
import pickle as pkl
import numpy as np
import re
import datetime

import matplotlib.pyplot as plt
import json

np.random.seed(1)

In [2]:
ACCESS_KEY='du3wAsX0uzEBBn4AdkFpCX4ZTXvSWFVG48GA4jbT'
SECRET_KEY='WA0NJOX0AFAY4K1J92XZ'

wr.config.s3_endpoint_url = 'https://obs.ru-moscow-1.hc.sbercloud.ru'

boto3.setup_default_session(
    region_name='ru-moscow',
    aws_access_key_id=SECRET_KEY,
    aws_secret_access_key=ACCESS_KEY,
)

In [4]:
#
path_catalogue = 's3://ml-okko-object-storage/catalogue/films_urls_2.csv'
catalogue = wr.s3.read_csv(path_catalogue, encoding="utf-16")

In [5]:
catalogue

,element_uid,name,f_cover_640;
0,0001d96c-2ab5-4a31-a727-3f489af70f8c,Просто вкусно. Сезон 1. Серия 306,https://okko.tv/static/42fc/7e31/4077/4987/a56...
1,00049d7c-6ce6-47d1-b957-5c27e8221939,Кот Саймона. Сезон 1. Серия 71,https://okko.tv/static/ef39/f04d/93cb/44d3/81a...
2,0009995a-605d-4958-afac-2457b8445d2f,Гриффины. Сезон 7,https://okko.tv/static/05ba/df91/cfa7/4e3a/9d5...
3,0009ecf0-8d42-4e21-add6-c50763d971b0,Обет на крови. Сезон 1. Серия 3,https://okko.tv/static/36a8/cf72/97da/4386/970...
4,000d7654-b9f1-4ea9-b2c7-42bfa3d1a35b,Comedy Club. Сезон 12. Серия 28,https://okko.tv/static/bf49/44c9/a24f/4f9d/a38...
...,...,...,...
179106,4ae91546-963e-4223-b173-4599a85f5c2d,Встреча с мечтой. Сезон 1. Серия 1,https://okko.tv/static/0b78/7eaa/dee5/4837/9b9...
179107,4e594869-f6cf-4979-9cc0-cdc3e57bd7dd,Знаки,https://okko.tv/static/e119/b905/0fb8/49b3/bd7...
179108,ffd64826-8020-4046-a7c9-2ce68d697a9b,Встреча с мечтой,https://okko.tv/static/b2b5/217d/6cc3/448e/91f...
179109,5e4e9605-8f9d-40ee-9e5f-40a2606732d4,Долина Йосемити,https://okko.tv/static/30a8/6841/4fad/4124/916...


In [9]:
catalogue['f_cover_640;'][4]

'https://okko.tv/static/bf49/44c9/a24f/4f9d/a386/1a747736f017/output_640x360.jpg;'

### **Вывод:**  

Обложки довольно плохие и не передают суть фильма/сериала

### New covers

In [5]:
path_covers = 's3://okko-develop/covers/_resultsCoverURL.json'
covers = wr.s3.read_json(path_covers).T

In [6]:
covers

,type,alias,name,url
42c5cbc6-7a44-4f7b-a620-d86f43290d1d,MOVIE,annie-leibovitz-life-through-a-lens,"Энни Лейбовиц: Жизнь, увиденная через объектив",https://static.okko.tv/images/v2/6238724
04579553-398a-4aa9-a672-e841ed4a6380,MOVIE,miss-congeniality-2-armed-fabulous,Мисс Конгениальность 2: Прекрасна и опасна,https://static.okko.tv/images/v2/16549415
e785baa6-f175-42b4-9e16-4319ac7991d5,MOVIE,star-trek-v-the-final-frontier,Звёздный путь 5: Последний рубеж,https://static.okko.tv/images/v2/16985871
f3921ca6-7441-46cc-a83a-c6beee368bf0,MOVIE,just-in-time-for-christmas,Как раз под Рождество,https://static.okko.tv/images/v2/8981988
5197752b-c863-4858-a12e-76d65c0a93e5,MOVIE,american-psycho-ii-all-american-girl,Американский психопат 2: Стопроцентная американка,https://static.okko.tv/images/v2/14768295
...,...,...,...,...
0c801d5e-92dd-4c43-b383-de2a8d572655,MP_MOVIE,manon-lescaut,Манон Леско,https://static.okko.tv/images/v2/13990304
ec8aab94-8f38-4547-8450-37560dc9e976,MP_MOVIE,rco-ivan-fischer-conducts-beethoven-the-comple...,"RCO: Все симфонии Бетховена, дирижирует Иван Ф...",https://static.okko.tv/images/v2/14569979
3460b114-e135-482e-bc50-f5d47f4309eb,MP_MOVIE,dgtl,Электронный музыкальный фестиваль DGTL,https://static.okko.tv/images/v2/11802993
59239ca8-593c-40b2-b0aa-d0999f20ecdd,MP_MOVIE,586e8ec6-a8c5-4c6c-9c6f-6444f4e06fa6,delete,NaN


In [8]:
covers['url'][5]

'https://static.okko.tv/images/v2/16561907'

Эти обложки более менее норм